In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np

In [64]:
list_of_urls = [
    'http://www.boxofficemojo.com/yearly/chart/?yr=2014&p=.htm',
    'http://www.boxofficemojo.com/yearly/chart/?yr=2013&p=.htm',
    'http://www.boxofficemojo.com/yearly/chart/?yr=2012&p=.htm',
    'http://www.boxofficemojo.com/yearly/chart/?yr=2011&p=.htm',
    'http://www.boxofficemojo.com/yearly/chart/?yr=2010&p=.htm',
    'http://www.boxofficemojo.com/yearly/chart/?yr=2009&p=.htm',
]

In [3]:
url = 'http://www.boxofficemojo.com/yearly/chart/?yr=2015&p=.htm'
response = requests.get(url)
soup = BeautifulSoup(response.text)

In [4]:
table = soup.find_all('table')[3]

In [5]:
movies = table.find_all('tr')

In [39]:
df = pd.DataFrame(columns=('title', 'studio', 'total_gross','theaters', 'opening','open_theaters', 'open', 'close', 'intlurl'))

In [7]:
testdata = movies[6].find_all('td')

In [40]:
for row in movies[6:106]:
    row = row.find_all('td')
    row_list = [item.get_text() for item in row][1:]
    # find the url linking to movie's page
    intl = row[1].find('a')['href']
    # add domain
    intl = 'http://www.boxofficemojo.com' + intl
    # insert '?page=intl&' to request intl page instead of main page
    intl = intl.split('?')[0] + '?page=intl&' + intl.split('?')[1]
    row_list.append(intl)
    row_s = pd.Series(row_list, index=df.columns)
    df = df.append(row_s, ignore_index=True)

In [45]:
df_BACKUP = df

In [56]:
def get_movies_with_urls(url_list):
    df_merge = df
    for url in list_of_urls:
        response = requests.get(url)
        soup = BeautifulSoup(response.text)
        table = soup.find_all('table')[3]
        movies = table.find_all('tr')
        tempdf = pd.DataFrame(columns=('title', 'studio', 'total_gross','theaters', 'opening','open_theaters', 'open', 'close', 'intlurl'))
        testdata = movies[6].find_all('td')
        for row in movies[6:106]:
            row = row.find_all('td')
            row_list = [item.get_text() for item in row][1:]
            # find the url linking to movie's page
            intl = row[1].find('a')['href']
            # add domain
            intl = 'http://www.boxofficemojo.com' + intl
            # insert '?page=intl&' to request intl page instead of main page
            intl = intl.split('?')[0] + '?page=intl&' + intl.split('?')[1]
            row_list.append(intl)
            row_s = pd.Series(row_list, index=tempdf.columns)
            tempdf = tempdf.append(row_s, ignore_index=True)
        df_merge = df_merge.append(tempdf)
    return df_merge

In [59]:
movie_urls = get_movies_with_urls(list_of_urls)

In [60]:
def get_movie_urls():
    return movie_urls

In [62]:
movie_df = pd.DataFrame(columns=('title', 'dtg', 'dist', 'release', 'genre', 'runtime', 'budget', 'rating', 'foreign', 'foreign_dist', 'foreign_release', 'foreign_opening', 'foreign_pct', 'foreign_total', 'foreign_asof', 'hungary', 'hun_dist', 'hun_release', 'hun_opening', 'hun_pct', 'hun_total', 'hun_asof'))

In [75]:
movie_urls_reindex = movie_urls.reindex()

In [77]:
movie_urls_reindex = movie_urls.reset_index()

In [80]:
foreign_urls = list(movie_urls_reindex['intlurl'])

['http://www.boxofficemojo.com/movies/?page=intl&id=starwars7.htm',
 'http://www.boxofficemojo.com/movies/?page=intl&id=jurassicpark4.htm',
 'http://www.boxofficemojo.com/movies/?page=intl&id=avengers2.htm',
 'http://www.boxofficemojo.com/movies/?page=intl&id=pixar2014.htm',
 'http://www.boxofficemojo.com/movies/?page=intl&id=fast7.htm',
 'http://www.boxofficemojo.com/movies/?page=intl&id=minions.htm',
 'http://www.boxofficemojo.com/movies/?page=intl&id=hungergames4.htm',
 'http://www.boxofficemojo.com/movies/?page=intl&id=scott2016.htm',
 'http://www.boxofficemojo.com/movies/?page=intl&id=cinderella2015.htm',
 'http://www.boxofficemojo.com/movies/?page=intl&id=bond24.htm']

In [84]:
def setup(site, movie_data):
    movie_title = ""
    movie_title = site.find('title').text.split(" (")[0]
    movie_data.append(movie_title)
    page_header = site.find_all('td')[4]
    international = site.find_all('table')[7]
    countries = international.find_all('tr')
    return movie_data, countries, page_header

In [85]:
def get_header_value(table, field_name):
    '''Grab a value from boxofficemojo HTML
    
    Takes a string attribute of a movie on the page and
    returns the string in the next sibling object
    (the value for that attribute)
    or None if nothing is found.
    '''
    obj = table.find(text=re.compile(field_name))
    # this works for most of the values
    next_sibling = obj.findNextSibling()
    if next_sibling:
        return next_sibling.text 
    else:
        return ''

In [86]:
def header(table, list_to_append):
    dtg = get_header_value(table,'Domestic Total')
    dist = get_header_value(table,'Distributor')
    release = get_header_value(table, 'Release Date')
    genre = get_header_value(table, 'Genre')
    runtime = get_header_value(table,'Runtime')
    budget = get_header_value(table, 'Budget')
    rating = get_header_value(table,'MPAA Rating')
    list_to_append.append(dtg)
    list_to_append.append(dist)
    list_to_append.append(release)
    list_to_append.append(genre)
    list_to_append.append(runtime)
    list_to_append.append(budget)
    list_to_append.append(rating)

In [87]:
# do this for foreign_total and for Hungary
# e.g. get_international('Hungary', hungary)
def get_international(country, country_list, country_table):
    country_data = ''
    for row in country_table:
        data = row.find_all('td')
        if data[0].find(text=country):
            country_data = row
    #get country name, insert as first element to list
    country_list.append(country_data.find('a').contents[0])
    columns = country_data.find_all('td')[1:]
    for column in columns:
        country_list.append(column.get_text())
    return country_list

In [88]:
def get_data(url):
    # return row of df if it exists
    # break out of that function but continue for loop if it does not
    response = requests.get(url)
    page = BeautifulSoup(response.text)
    
    movie_data = []
    foreign_total = []
    hungary = []
    
    if page.find_all(text="Hungary"):
        movie_data, countries, page_header = setup(page, movie_data)
        header(page_header, movie_data)
        get_international('FOREIGN TOTAL', foreign_total, countries)
        get_international('Hungary', hungary, countries)
        movie_row = movie_data + foreign_total + hungary
        return movie_row

In [89]:
for url in foreign_urls:
    row = get_data(url)
    if row:
        row_s = pd.Series(row, index=movie_df.columns)
        movie_df = movie_df.append(row_s, ignore_index=True)

In [94]:
movie_df.head(50)

,title,dtg,dist,release,genre,runtime,budget,rating,foreign,foreign_dist,...,foreign_pct,foreign_total,foreign_asof,hungary,hun_dist,hun_release,hun_opening,hun_pct,hun_total,hun_asof
0,Star Wars: The Force Awakens,,Buena Vista,"December 18, 2015",Sci-Fi Fantasy,2 hrs. 16 min.,$200 million,PG-13,FOREIGN TOTAL,-,...,26.5%,"$1,060,700,000",1/24/16,Hungary,Forum Films,12/18/15,"$1,508,298",24.4%,"$6,181,662",1/24/16
1,Jurassic World,"$652,270,625",Universal,"June 12, 2015",Sci-Fi Horror,2 hrs. 4 min.,$150 million,PG-13,FOREIGN TOTAL,-,...,31.1%,"$1,016,714,301",11/11/15,Hungary,UIP,6/11/15,"$604,551",29.5%,"$2,051,915",7/5/15
2,Avengers: Age of Ultron,"$459,005,868",Buena Vista,"May 1, 2015",Action / Adventure,2 hrs. 21 min.,$250 million,PG-13,FOREIGN TOTAL,-,...,21.3%,"$946,029,899",N/A,Hungary,Forum Films,5/7/15,"$807,040",40.7%,"$1,980,532",6/21/15
3,Inside Out,"$356,461,711",Buena Vista,"June 19, 2015",Animation,1 hrs. 34 min.,$175 million,PG,FOREIGN TOTAL,-,...,-,"$499,668,421",12/29/15,Hungary,Forum Films,6/25/15,"$296,617",24.3%,"$1,218,401",9/27/15
4,Furious 7,"$353,007,020",Universal,"April 3, 2015",Action,2 hrs. 17 min.,$190 million,PG-13,FOREIGN TOTAL,-,...,21.6%,"$1,162,040,651",10/28/15,Hungary,UIP,4/2/15,"$661,897",33.8%,"$1,958,035",5/24/15
5,Minions,"$336,045,770",Universal,"July 10, 2015",Animation,1 hrs. 31 min.,$74 million,PG,FOREIGN TOTAL,-,...,1.5%,"$821,229,247",11/8/15,Hungary,UIP,7/9/15,-,-,"$3,493,481",10/11/15
6,The Hunger Games: Mockingjay - Part 2,,Lionsgate,"November 20, 2015",Action / Adventure,2 hrs. 17 min.,$160 million,PG-13,FOREIGN TOTAL,-,...,39%,"$370,861,252",1/24/16,Hungary,Forum,11/19/15,"$519,856",41.8%,"$1,243,577",12/27/15
7,The Martian,,Fox,"October 2, 2015",Sci-Fi,2 hrs. 21 min.,$108 million,PG-13,FOREIGN TOTAL,-,...,12%,"$370,553,098",1/12/16,Hungary,InterCom,10/1/15,"$357,846",24.9%,"$1,438,744",12/6/15
8,Cinderella,"$201,151,353",Buena Vista,"March 13, 2015",Fantasy,1 hrs. 52 min.,$95 million,PG,FOREIGN TOTAL,-,...,-,"$341,535,384",N/A,Hungary,Forum Films,3/19/15,"$121,131",23.4%,"$518,592",5/10/15
9,Spectre,,Sony / Columbia,"November 6, 2015",Action,2 hrs. 28 min.,$245 million,PG-13,FOREIGN TOTAL,-,...,4.5%,"$678,200,000",1/24/16,Hungary,Forum,11/5/15,"$474,075",35.5%,"$1,333,699",12/20/15


In [95]:
movie_df_BACKUP = movie_df

In [96]:
movie_df2 = movie_df

In [97]:
MOVIEDF = movie_df

In [100]:
movie_df.to_csv('moviedf.csv', index=False)